In [10]:
import pandas as pd
import numpy as np
import re

In [11]:
!unzip "/content/drive/MyDrive/Colab Notebooks/DATASET_FOR_GPT2_FT.zip"

Archive:  /content/drive/MyDrive/Colab Notebooks/DATASET_FOR_GPT2_FT.zip
replace Articles.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Articles.csv            


In [12]:
def cleaning(s):
    s = str(s)  # Ensure the input is a string
    s = re.sub(r'\s\W', ' ', s)  # Replace whitespace followed by a non-word character with a space
    s = re.sub(r'\W,\s', ' ', s)  # Replace a non-word character followed by a comma and whitespace with a space
    s = re.sub(r"\d+", "", s)  # Remove all digits
    s = re.sub(r'\s+', ' ', s)  # Replace multiple whitespace characters with a single space
    s = re.sub(r'[!@#$_]', '', s)  # Remove specific special characters (!, @, #, $, _)
    s = s.replace("co", "")  # Remove occurrences of the substring "co"
    s = s.replace("https", "")  # Remove occurrences of the substring "https"
    s = s.replace("[\w*", " ")  # Replace the literal string "[\w*" with a space
    return s  # Return the cleaned string


<>:10: SyntaxWarning: invalid escape sequence '\w'
<>:10: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-258771006.py:10: SyntaxWarning: invalid escape sequence '\w'
  s = s.replace("[\w*", " ")  # Replace the literal string "[\w*" with a space


In [13]:
df = pd.read_csv("/content/Articles.csv", encoding="ISO-8859-1")
df = df.dropna()

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
df.sample(5)

,Article,Date,Heading,NewsType
594,DUBAI: Iran´s oil exports to South Korea have ...,5/1/2016,Iran oil exports to South Korea rise after san...,business
2006,strong>LAHORE: Pakistan´s top Test batsman You...,6/17/2016,Younis Khan English attack Lords retur,sports
368,Singapore: Oil prices held above $43 a barrel ...,11/26/2015,oil prices up in asi,business
875,strong>ISLAMABAD: The Federal Board of Revenue...,8/2/2016,FBR issues immovable property valuation tables...,business
26,KARACHI: State Bank of Pakistan (SBP) governor...,1/29/2015,sbp warns islamic banks to be more equitab,business


In [16]:
text_data = open("/content/Articles.txt", 'w')
for idx, item in df.iterrows():
  article = cleaning(item["Article"])
  text_data.write(article)
text_data.close()

In [17]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [18]:
# Loading Dataset

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [19]:
# Loading Data Collator

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [20]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [21]:
# Setting parameters

train_file_path = "/content/Articles.txt"
model_name = 'gpt2'
output_dir = '/content/results'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 3.0
save_steps = 500

In [22]:
# Training
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: csd1-19-54 (csd1-19-54-octaloop) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,3.662100
1000,3.398300
1500,3.166200
2000,3.119700
2500,3.015400
3000,2.991900


In [23]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [24]:
# Testing

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "/content/results"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [26]:
sequence = input()
max_len = int(input())
generate_text(sequence, max_len)

oil pricing
70
oil pricing and the impact of new ntracts.</strongThe mpany, which has so far received little from the government, will take stock on Monday on how it will manage its cash flow and plans for financing a -month ntract.Under the loan terms, the mpany will finance at least one year's interest rate reduction,
